In [ ]:
from google.colab import drive
drive.mount('/content/drive')
!pip install jellyfish
!pip install unidecode
!pip install samplings
import os


Mounted at /content/drive
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 356.9/356.9 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.8/235.8 kB 7.5 MB/s eta 0:00:00


In [ ]:
os.chdir('/content/drive/MyDrive/tunesformer-main/')  # Cambia el directorio
print(os.getcwd())    # Muestra el directorio actual, debe salir /content

/content/drive/MyDrive/tunesformer-main


In [ ]:
subprocess.Popen('generate.py', stdout=subprocess.PIPE, shell=True)

<Popen: returncode: None args: 'generate.py'>

In [ ]:
#BatchConverter.py
import os
import math
import subprocess
from tqdm import trange
from multiprocessing import Pool

class suppress_stdout_stderr(object):
    '''
    A context manager for doing a "deep suppression" of stdout and stderr in
    Python, i.e. will suppress all print, even if the print originates in a
    compiled C/Fortran sub-function.
       This will not suppress raised exceptions, since exceptions are printed
    to stderr just before a script exits, and after the context manager has
    exited (at least, I think that is why it lets exceptions through).

    '''
    def __init__(self):
        # Open a pair of null files
        self.null_fds = [os.open(os.devnull, os.O_RDWR) for x in range(2)]
        # Save the actual stdout (1) and stderr (2) file descriptors.
        self.save_fds = (os.dup(1), os.dup(2))

    def __enter__(self):
        # Assign the null pointers to stdout and stderr.
        os.dup2(self.null_fds[0], 1)
        os.dup2(self.null_fds[1], 2)

    def __exit__(self, *_):
        # Re-assign the real stdout/stderr back to (1) and (2)
        os.dup2(self.save_fds[0], 1)
        os.dup2(self.save_fds[1], 2)
        # Close the null files
        os.close(self.null_fds[0])
        os.close(self.null_fds[1])

def run_filter(lines):
    score = ""
    for line in lines.replace("\r", "").split("\n"):
        if line[:2] in ['A:', 'B:', 'C:', 'D:', 'F:', 'G', 'H:', 'N:', 'O:', 'R:', 'r:', 'S:', 'T:', 'V:', 'W:', 'w:', 'X:', 'Z:'] \
        or line=='\n' \
        or line.startswith('%'):
            continue
        else:
            if '%' in line:
                line = line.split('%')
                bar = line[-1]
                line = ''.join(line[:-1])
            score += line + '\n'

    return score.strip()

cmd = 'cd '+os.getcwd()
output = os.popen(cmd).read()
cmd = 'python data_curation/xml2abc.py -m 2 -c 6 -x '

def convert_abc(file_list):
    for file_idx in trange(len(file_list)):
        file = file_list[file_idx]
        filename = file.split('/')[-1]
        try:
            p = subprocess.Popen(cmd+'"'+file+'"', stdout=subprocess.PIPE, shell=True)
            result = p.communicate()
            output = result[0].decode('utf-8')
            #print(output)
            output = run_filter(output)
            if output=='':
                continue
            else:
                with open('data_curation/abc/'+filename.split('.')[-2]+'.txt', 'w', encoding='utf-8') as f:
                    f.write(output)
        except:
            pass

if __name__ == '__main__':

    file_list = []
    abc_list = []

    # traverse folder
    for root, dirs, files in os.walk('data_curation/xmls'):
        for file in files:
            filename = os.path.join(root, file)
            file_list.append(filename)

    # convert_abc(file_list)
    file_lists = []
    for i in range(os.cpu_count()):
        start_idx = int(math.floor(i*len(file_list)/os.cpu_count()))
        end_idx = int(math.floor((i+1)*len(file_list)/os.cpu_count()))
        file_lists.append(file_list[start_idx:end_idx])

    cnt = 0

    for file_list in file_lists:
        cnt += len(file_list)

    pool = Pool(processes=os.cpu_count())
    pool.map(convert_abc, file_lists)

100%|██████████| 35/35 [00:30<00:00,  1.14it/s]


In [ ]:
import re
import os
import json
import jellyfish
from tqdm import tqdm
from unidecode import unidecode


In [ ]:
import os
import re
import json
import hashlib
from tqdm import tqdm

def clean_metadata(content):
    """Limpia metadata duplicada y mantiene solo la información esencial"""
    lines = content.split('\n')
    metadata = {}

    for line in lines:
        if len(line) > 1 and line[0].isalpha() and line[1] == ':':
            key = line[0]
            value = line[2:].strip()

            # Mantener solo el primer valor para evitar duplicados
            if key not in metadata:
                metadata[key] = value

    # Orden específico para metadata esencial
    essential_keys = ['X', 'T', 'C', 'L', 'M', 'K']
    clean_metadata = []

    for key in essential_keys:
        if key in metadata:
            clean_metadata.append(f"{key}:{metadata[key]}")

    return '\n'.join(clean_metadata)

def generate_piece_id(content):
    """Genera un ID único para la pieza basado en el contenido"""
    # Usar título y compositor para generar ID único
    title = ""
    composer = ""

    lines = content.split('\n')
    for line in lines:
        if line.startswith('T:'):
            title = line[2:].strip()
            break
    for line in lines:
        if line.startswith('C:'):
            composer = line[2:].strip()
            break

    # Generar hash corto del contenido combinado
    combined = f"{title}_{composer}"
    hash_obj = hashlib.md5(combined.encode())
    return hash_obj.hexdigest()[:8]

def extract_voice_info(content):
    """Extrae información específica de cada voz"""
    voices = {}
    lines = content.split('\n')
    current_voice = None

    for line in lines:
        # Detectar definición de voz
        if line.startswith('V:'):
            voice_match = re.search(r'V:(\d+)', line)
            if voice_match:
                voice_num = voice_match.group(1)
                current_voice = voice_num

                voices[voice_num] = {
                    'voice_id': f"V{voice_num}",
                    'content': [],
                    'definition_line': line
                }

        # Agregar contenido musical a la voz actual
        elif current_voice and current_voice in voices:
            # Solo agregar líneas con contenido musical
            if line.strip() and not (len(line) > 1 and line[0].isalpha() and line[1] == ':'):
                if not line.startswith('V:') and not line.startswith('%'):
                    voices[current_voice]['content'].append(line)

    return voices

def count_measures_in_voice(voice_content):
    """Cuenta compases en el contenido de una voz específica"""
    if not voice_content:
        return 0
    content = ' '.join(voice_content)
    # Contar barras de compás
    bar_count = content.count('|')
    return max(1, bar_count)

def split_voice_content(voice_content, max_measures=32):
    """Divide el contenido de una voz en segmentos manejables"""
    if not voice_content:
        return [voice_content]

    measures_count = count_measures_in_voice(voice_content)

    if measures_count <= max_measures:
        return [voice_content]

    # Dividir por líneas y contar medidas acumulativas
    segments = []
    current_segment = []
    current_measures = 0

    for line in voice_content:
        line_measures = line.count('|')

        # Si agregar esta línea excede el límite, iniciar nuevo segmento
        if current_measures + line_measures > max_measures and current_segment:
            segments.append(current_segment[:])
            current_segment = [line]
            current_measures = line_measures
        else:
            current_segment.append(line)
            current_measures += line_measures

    # Agregar último segmento
    if current_segment:
        segments.append(current_segment)

    return segments

def generate_voice_control_code(voice_id, segment_num, total_segments, measures, key, meter, piece_id):
    """Genera código de control específico para una voz con información de sincronización"""
    control_parts = [
        f"PIECE:{piece_id}",
        f"VOICE:{voice_id}",
        f"MEASURES:{measures}",
        f"KEY:{key}",
        f"METER:{meter}"
    ]

    # Agregar información de segmentación si aplica
    if total_segments > 1:
        control_parts.append(f"SEGMENT:{segment_num}/{total_segments}")

    # Clasificación por complejidad
    if measures <= 16:
        control_parts.append("COMPLEXITY:SIMPLE")
    elif measures <= 32:
        control_parts.append("COMPLEXITY:MEDIUM")
    else:
        control_parts.append("COMPLEXITY:COMPLEX")

    return " ".join(control_parts) + "\n"

def process_multi_voice_abc(content):
    """Procesa archivo ABC con múltiples voces y genera elementos separados pero sincronizados"""
    results = []

    # Generar ID de pieza
    piece_id = generate_piece_id(content)

    # Limpiar metadata (solo una vez para toda la pieza)
    clean_meta = clean_metadata(content)

    # Extraer información básica
    key = extract_key_signature(content)
    meter = extract_time_signature(content)

    # Extraer voces
    voices = extract_voice_info(content)

    if not voices:
        # Si no hay voces explícitas, tratar como voz única
        return process_single_voice_abc(content)

    print(f"Encontradas {len(voices)} voces: {list(voices.keys())}")

    # Dividir todas las voces en el mismo número de segmentos para mantener sincronización
    all_voice_segments = {}
    max_segments = 0

    # Primero dividir cada voz y encontrar el máximo número de segmentos
    for voice_num, voice_data in voices.items():
        voice_content = voice_data['content']
        if voice_content:
            segments = split_voice_content(voice_content, max_measures=32)
            all_voice_segments[voice_num] = segments
            max_segments = max(max_segments, len(segments))

    # Ahora generar elementos sincronizados
    for segment_idx in range(max_segments):
        segment_voices = []

        # Recopilar todas las voces para este segmento
        for voice_num, voice_data in voices.items():
            if voice_num in all_voice_segments and segment_idx < len(all_voice_segments[voice_num]):
                voice_id = voice_data['voice_id']
                segment_content = all_voice_segments[voice_num][segment_idx]

                if segment_content:
                    segment_measures = count_measures_in_voice(segment_content)

                    # Generar código de control
                    control_code = generate_voice_control_code(
                        voice_id, segment_idx+1, max_segments, segment_measures, key, meter, piece_id
                    )

                    # Construir notación ABC para este segmento
                    # IMPORTANTE: Metadata solo en V1 del primer segmento
                    abc_lines = []

                    if voice_id == "V1" and segment_idx == 0:
                        # Solo la primera voz del primer segmento lleva metadata completa
                        abc_lines.append(clean_meta)
                    else:
                        # Otras voces/segmentos solo llevan el número X
                        abc_lines.append("X:1")

                    abc_lines.append(f"V:{voice_num}")
                    abc_lines.extend(segment_content)

                    abc_notation = '\n'.join(abc_lines)

                    results.append({
                        'control code': control_code,
                        'abc notation': abc_notation
                    })

    return results

def process_single_voice_abc(content):
    """Procesa archivo ABC de una sola voz"""
    # Generar ID de pieza
    piece_id = generate_piece_id(content)

    # Limpiar metadata
    clean_meta = clean_metadata(content)

    # Extraer información básica
    key = extract_key_signature(content)
    meter = extract_time_signature(content)

    # Separar metadata del contenido musical
    lines = content.split('\n')
    musical_lines = []

    for line in lines:
        # Solo líneas con contenido musical
        if line.strip() and not (len(line) > 1 and line[0].isalpha() and line[1] == ':'):
            if not line.startswith('%'):
                musical_lines.append(line)

    if not musical_lines:
        return []

    # Dividir en segmentos
    segments = split_voice_content(musical_lines, max_measures=32)
    results = []

    for i, segment in enumerate(segments):
        if not segment:
            continue

        segment_measures = count_measures_in_voice(segment)

        # Generar código de control
        control_code = generate_voice_control_code(
            'V1', i+1, len(segments), segment_measures, key, meter, piece_id
        )

        # Construir notación ABC
        abc_lines = []

        if i == 0:
            # Solo el primer segmento lleva metadata completa
            abc_lines.append(clean_meta)
        else:
            abc_lines.append("X:1")

        abc_lines.extend(segment)

        abc_notation = '\n'.join(abc_lines)

        results.append({
            'control code': control_code,
            'abc notation': abc_notation
        })

    return results

def extract_key_signature(content):
    """Extrae la tonalidad de la pieza"""
    lines = content.split('\n')
    for line in lines:
        if line.startswith('K:'):
            key = line[2:].strip()
            key = re.sub(r'\s+.*', '', key)
            return key
    return "C"

def extract_time_signature(content):
    """Extrae el compás de la pieza"""
    lines = content.split('\n')
    for line in lines:
        if line.startswith('M:'):
            meter = line[2:].strip()
            return meter
    return "4/4"

def process_abc_file(filename):
    """Procesa un archivo ABC individual"""
    try:
        with open(filename, 'r', encoding='utf-8', errors='ignore') as f:
            content = f.read()

        if not content.strip():
            return []

        # Detectar si tiene múltiples voces
        voice_pattern = re.findall(r'^V:\d+', content, re.MULTILINE)
        if voice_pattern:
            return process_multi_voice_abc(content)
        else:
            return process_single_voice_abc(content)

    except Exception as e:
        print(f"Error procesando {filename}: {e}")
        return []

if __name__ == "__main__":
    data = []
    filenames = []

    # Buscar archivos ABC
    possible_dirs = ['data_curation/abc_files']
    abc_dir = None

    for dir_path in possible_dirs:
        if os.path.exists(dir_path):
            abc_dir = dir_path
            break

    if abc_dir is None:
        print("No se encontró directorio de archivos ABC")
        exit(1)

    print(f"Usando directorio: {abc_dir}")

    # Recopilar archivos
    for dirpath, dirlist, filelist in os.walk(abc_dir):
        for this_file in filelist:
            if this_file.endswith(('.abc', '.txt')):
                filename = os.path.join(dirpath, this_file)
                filenames.append(filename)

    print(f"Archivos encontrados: {len(filenames)}")

    # Procesar todos los archivos
    processed_files = 0
    total_segments = 0

    for filename in tqdm(filenames):
        results = process_abc_file(filename)
        if results:
            data.extend(results)
            processed_files += 1
            total_segments += len(results)

    print(f"Archivos procesados: {processed_files}")
    print(f"Segmentos generados: {total_segments}")

    # Estadísticas
    voice_stats = {}
    piece_stats = {}
    complexity_stats = {}

    for item in data:
        # Estadísticas de voces
        voice_match = re.search(r'VOICE:(\w+)', item['control code'])
        if voice_match:
            voice = voice_match.group(1)
            voice_stats[voice] = voice_stats.get(voice, 0) + 1

        # Estadísticas de piezas
        piece_match = re.search(r'PIECE:(\w+)', item['control code'])
        if piece_match:
            piece = piece_match.group(1)
            piece_stats[piece] = piece_stats.get(piece, 0) + 1

        # Estadísticas de complejidad
        complexity_match = re.search(r'COMPLEXITY:(\w+)', item['control code'])
        if complexity_match:
            complexity = complexity_match.group(1)
            complexity_stats[complexity] = complexity_stats.get(complexity, 0) + 1

    print(f"\nSegmentos totales generados: {len(data)}")
    print("Distribución por voces:", voice_stats)
    print(f"Número de piezas únicas: {len(piece_stats)}")
    print("Distribución por complejidad:", complexity_stats)

    # Guardar datos
    output_file = 'data_curation/data_voice_synchronized.json'
    os.makedirs(os.path.dirname(output_file), exist_ok=True)

    with open(output_file, 'w', encoding='utf-8') as f:
        json.dump(data, f, ensure_ascii=False, indent=2)

    print(f"Datos guardados en: {output_file}")

    # Mostrar ejemplos de sincronización
    if len(data) > 0:
        print("\nEjemplo de sincronización - misma pieza, diferentes voces:")
        first_piece_id = None

        for item in data:
            piece_match = re.search(r'PIECE:(\w+)', item['control code'])
            if piece_match:
                piece_id = piece_match.group(1)
                if first_piece_id is None:
                    first_piece_id = piece_id

                if piece_id == first_piece_id:
                    print(f"\nControl: {item['control code'].strip()}")
                    print(f"ABC (primeros 150 chars): {item['abc notation'][:150]}")

                    # Mostrar solo primeros 3 elementos de la misma pieza
                    voice_match = re.search(r'VOICE:(\w+)', item['control code'])
                    if voice_match and voice_match.group(1) == 'V3':
                        break

Usando directorio: data_curation/abc_files
Archivos encontrados: 70


 30%|███       | 21/70 [00:00<00:00, 201.10it/s]

Encontradas 4 voces: ['1', '2', '3', '4']
Encontradas 5 voces: ['1', '2', '3', '4', '5']
Encontradas 4 voces: ['1', '2', '3', '4']
Encontradas 4 voces: ['1', '2', '3', '4']
Encontradas 4 voces: ['1', '2', '3', '4']
Encontradas 4 voces: ['1', '2', '3', '4']
Encontradas 4 voces: ['1', '2', '3', '4']
Encontradas 4 voces: ['1', '2', '3', '4']
Encontradas 5 voces: ['1', '2', '3', '4', '5']
Encontradas 4 voces: ['1', '2', '3', '4']
Encontradas 4 voces: ['1', '2', '3', '4']
Encontradas 4 voces: ['1', '2', '3', '4']
Encontradas 4 voces: ['1', '2', '3', '4']
Encontradas 5 voces: ['1', '2', '3', '4', '5']
Encontradas 8 voces: ['1', '2', '3', '4', '5', '6', '7', '8']
Encontradas 4 voces: ['1', '2', '3', '4']
Encontradas 4 voces: ['1', '2', '3', '4']
Encontradas 7 voces: ['1', '2', '3', '4', '5', '6', '7']
Encontradas 4 voces: ['1', '2', '3', '4']
Encontradas 5 voces: ['1', '2', '3', '4', '5']
Encontradas 6 voces: ['1', '2', '3', '4', '5', '6']
Encontradas 5 voces: ['1', '2', '3', '4', '5']
Encont

100%|██████████| 70/70 [00:00<00:00, 176.13it/s]

Encontradas 4 voces: ['1', '2', '3', '4']
Encontradas 4 voces: ['1', '2', '3', '4']
Encontradas 8 voces: ['1', '2', '3', '4', '5', '6', '7', '8']
Encontradas 4 voces: ['1', '2', '3', '4']
Encontradas 4 voces: ['1', '2', '3', '4']
Encontradas 4 voces: ['1', '2', '3', '4']
Encontradas 7 voces: ['1', '2', '3', '4', '5', '6', '7']
Encontradas 7 voces: ['1', '2', '3', '4', '5', '6', '7']
Encontradas 5 voces: ['1', '2', '3', '4', '5']
Encontradas 6 voces: ['1', '2', '3', '4', '5', '6']
Encontradas 4 voces: ['1', '2', '3', '4']
Encontradas 5 voces: ['1', '2', '3', '4', '5']
Encontradas 5 voces: ['1', '2', '3', '4', '5']
Encontradas 6 voces: ['1', '2', '3', '4', '5', '6']
Encontradas 4 voces: ['1', '2', '3', '4']
Encontradas 6 voces: ['1', '2', '3', '4', '5', '6']
Encontradas 5 voces: ['1', '2', '3', '4', '5']
Encontradas 7 voces: ['1', '2', '3', '4', '5', '6', '7']
Encontradas 5 voces: ['1', '2', '3', '4', '5']
Encontradas 4 voces: ['1', '2', '3', '4']
Encontradas 5 voces: ['1', '2', '3', '4'

Datos guardados en: data_curation/data_voice_synchronized.json

Ejemplo de sincronización - misma pieza, diferentes voces:

Control: PIECE:64df3190 VOICE:V1 MEASURES:27 KEY:Eb METER:2/2 SEGMENT:1/9 COMPLEXITY:MEDIUM
ABC (primeros 150 chars): X:1
T:String Quartet
C:Ludwig van Beethoven
L:1/8
M:2/2
K:Eb
V:1
"_sotto voce""^Poco Adagio" (e4 G3 A) | _d2 z4 z2 | (e4 G3 A) | _d8- | d6 =d2- | d4 (

Control: PIECE:64df3190 VOICE:V2 MEASURES:32 KEY:Eb METER:2/2 SEGMENT:1/9 COMPLEXITY:MEDIUM
ABC (primeros 150 chars): X:1
V:2
"_sotto voce" G4 E3 E | E2 z4 z2 | G4 E3 E | =E2 z2 b4- | b4 a4- | a4 (g2 fF) | (F4 A2 G2) | %7
 (G2 F) z B,2 B, z | C2 z4 z2 | z2"_cresc." (F

Control: PIECE:64df3190 VOICE:V3 MEASURES:31 KEY:Eb METER:2/2 SEGMENT:1/9 COMPLEXITY:MEDIUM
ABC (primeros 150 chars): X:1
V:3
"_sotto voce" B,8 (B,6 A,2) | G,4 z8 z4 | B,8 (B,6 A,2) | B,4 z8 =E4- | E8 F8- | (F4 _c4 B4 =C4) | %6
 (A,8 D,6 _E,2) | (=E,4 F,4 D,4 _E,4) | 


Clean json data


In [ ]:
import json
import re

def clean_abc_notation(abc_text):
    """Limpia notación ABC para facilitar el aprendizaje del modelo"""

    # Remover indicaciones interpretativas complejas pero mantener dinámicas básicas
    # Mantener: !p!, !f!, !mp!, !mf!
    # Remover: indicaciones de texto complejas
    abc_text = re.sub(r'"[^"]*"(?![fp]!)', '', abc_text)  # Remover texto entre comillas excepto dinámicas

    # Simplificar ornamentos complejos
    abc_text = re.sub(r'\{[^}]*\}', '', abc_text)  # Remover grace notes complejas

    # Limpiar comentarios de compás
    abc_text = re.sub(r'\s*%\d+', '', abc_text)  # Remover números de compás

    # Normalizar espacios
    abc_text = re.sub(r'\s+', ' ', abc_text)
    abc_text = re.sub(r'\s*\|\s*', '|', abc_text)  # Normalizar barras

    return abc_text.strip()

def validate_synchronization(data):
    """Valida que las voces estén sincronizadas correctamente"""
    pieces = {}

    # Agrupar por pieza y segmento
    for item in data:
        piece_match = re.search(r'PIECE:(\w+)', item['control code'])
        segment_match = re.search(r'SEGMENT:(\d+)/(\d+)', item['control code'])

        if piece_match and segment_match:
            piece_id = piece_match.group(1)
            segment_num = segment_match.group(1)
            total_segments = segment_match.group(2)

            key = f"{piece_id}_SEG{segment_num}"

            if key not in pieces:
                pieces[key] = []

            pieces[key].append(item)

    # Validar cada grupo
    issues = []
    for key, items in pieces.items():
        if len(items) > 1:
            # Extraer medidas de cada voz
            measures = []
            for item in items:
                measure_match = re.search(r'MEASURES:(\d+)', item['control code'])
                if measure_match:
                    measures.append(int(measure_match.group(1)))

            # Verificar si están sincronizadas (diferencia máxima de 2 compases)
            if measures and (max(measures) - min(measures)) > 2:
                issues.append(f"{key}: medidas desincronizadas {measures}")

    return issues

def filter_training_ready_data(input_file, output_file):
    """Filtra y limpia datos para entrenamiento"""

    with open(input_file, 'r', encoding='utf-8') as f:
        data = json.load(f)

    print(f"Datos originales: {len(data)} elementos")

    # Validar sincronización
    sync_issues = validate_synchronization(data)
    if sync_issues:
        print(f"Encontrados {len(sync_issues)} problemas de sincronización")
        for issue in sync_issues[:5]:  # Mostrar primeros 5
            print(f"  {issue}")

    # Limpiar notación ABC
    cleaned_data = []
    for item in data:
        cleaned_item = item.copy()
        cleaned_item['abc notation'] = clean_abc_notation(item['abc notation'])

        # Filtrar elementos muy cortos o muy largos
        abc_length = len(cleaned_item['abc notation'])
        if 50 <= abc_length <= 2000:  # Rango razonable
            cleaned_data.append(cleaned_item)

    print(f"Datos después de limpieza: {len(cleaned_data)} elementos")

    # Guardar datos limpios
    with open(output_file, 'w', encoding='utf-8') as f:
        json.dump(cleaned_data, f, ensure_ascii=False, indent=2)

    # Estadísticas finales
    voice_stats = {}
    for item in cleaned_data:
        voice_match = re.search(r'VOICE:(\w+)', item['control code'])
        if voice_match:
            voice = voice_match.group(1)
            voice_stats[voice] = voice_stats.get(voice, 0) + 1

    print(f"Distribución final por voces: {voice_stats}")
    print(f"Datos listos para entrenamiento guardados en: {output_file}")

    # Mostrar ejemplo limpio
    if cleaned_data:
        print(f"\nEjemplo de dato limpio:")
        print(f"Control: {cleaned_data[0]['control code'].strip()}")
        print(f"ABC (primeros 200 chars): {cleaned_data[0]['abc notation'][:200]}")

if __name__ == "__main__":
    input_file = 'data_curation/data_voice_synchronized.json'
    output_file = 'data_curation/data_training_ready.json'

    filter_training_ready_data(input_file, output_file)

Datos originales: 3033 elementos
Encontrados 218 problemas de sincronización
  64df3190_SEG1: medidas desincronizadas [27, 32, 31, 28, 29, 27, 29, 19, 30, 29, 26, 31, 30, 30, 32, 29, 30]
  64df3190_SEG2: medidas desincronizadas [30, 30, 31, 32, 30, 27, 30, 19, 29, 31, 31, 31, 30, 28, 27, 28, 31]
  64df3190_SEG3: medidas desincronizadas [31, 31, 32, 30, 30, 29, 32, 19, 29, 22, 22, 22, 28, 30, 27, 31, 30]
  64df3190_SEG4: medidas desincronizadas [28, 31, 29, 32, 29, 31, 29, 19, 32, 32, 17, 29, 29, 32, 28, 30, 30]
  64df3190_SEG5: medidas desincronizadas [29, 32, 29, 30, 29, 27, 30, 19, 28, 23, 30, 28, 32, 31, 22, 30, 32]
Datos después de limpieza: 3023 elementos
Distribución final por voces: {'V1': 568, 'V2': 625, 'V3': 628, 'V4': 610, 'V5': 315, 'V6': 157, 'V7': 88, 'V8': 32}
Datos listos para entrenamiento guardados en: data_curation/data_training_ready.json

Ejemplo de dato limpio:
Control: PIECE:64df3190 VOICE:V1 MEASURES:27 KEY:Eb METER:2/2 SEGMENT:1/9 COMPLEXITY:MEDIUM
ABC (primero

In [ ]:
import os
import time
import json
import torch
import random
from utils import *
from config import *
from tqdm import tqdm
from torch.cuda.amp import autocast, GradScaler
from torch.utils.data import Dataset, DataLoader
from transformers import GPT2Config, get_scheduler

if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

random.seed(42)
# Fix: Asegurar que batch_size sea al menos 1
batch_size = max(1, torch.cuda.device_count())
patchilizer = Patchilizer()

patch_config = GPT2Config(num_hidden_layers=PATCH_NUM_LAYERS,
                    max_length=PATCH_LENGTH,
                    max_position_embeddings=PATCH_LENGTH,
                    vocab_size=1)
char_config = GPT2Config(num_hidden_layers=CHAR_NUM_LAYERS,
                    max_length=PATCH_SIZE,
                    max_position_embeddings=PATCH_SIZE,
                    vocab_size=128)
model = TunesFormer(patch_config, char_config, share_weights=SHARE_WEIGHTS)

# print parameter number
print("Parameter Number: "+str(sum(p.numel() for p in model.parameters() if p.requires_grad)))

if torch.cuda.device_count() > 1:
    model = torch.nn.DataParallel(model)

scaler = GradScaler()
is_autocast = True
optimizer = torch.optim.AdamW(model.parameters(), lr=LEARNING_RATE)

def collate_batch(batch):
    input_patches = []

    for input_patch in batch:
        input_patches.append(input_patch.reshape(-1))

    input_patches = torch.nn.utils.rnn.pad_sequence(input_patches, batch_first=True, padding_value=0)

    return input_patches.to(device)

def split_data(data, eval_ratio=0.1):
    random.shuffle(data)
    split_idx = int(len(data)*eval_ratio)
    eval_set = data[:split_idx]
    train_set = data[split_idx:]
    return train_set, eval_set

class MyDataset(Dataset):
    def __init__(self, items):
        self.texts = []

        for item in tqdm(items):
            text = item['control code']+"\n".join(item['abc notation'].split('\n')[1:])
            input_patch =  patchilizer.encode(text, add_special_patches=True)
            input_patch = torch.tensor(input_patch)
            if torch.sum(input_patch)!=0:
                self.texts.append(input_patch)

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        return self.texts[idx]

# call model with a batch of input
def process_one_batch(batch):
    input_patches = batch

    loss = model(input_patches).loss

    return loss.mean()

# do one epoch for training
def train_epoch():
    tqdm_train_set = tqdm(train_set)
    total_train_loss = 0
    iter_idx = 1
    model.train()

    for batch in tqdm_train_set:
        try:
            if is_autocast:
                with autocast():
                    loss = process_one_batch(batch)
                if loss==None or torch.isnan(loss).item():
                    continue
                scaler.scale(loss).backward()
                scaler.step(optimizer)
                scaler.update()
            else:
                loss = process_one_batch(batch)
                if loss==None or torch.isnan(loss).item():
                    continue
                loss.backward()
                optimizer.step()
        except RuntimeError as exception:
            if "memory" in str(exception):
                print(str(exception))
                if hasattr(torch.cuda, 'empty_cache'):
                    torch.cuda.empty_cache()
                continue
            else:
                raise exception
        lr_scheduler.step()
        model.zero_grad(set_to_none=True)
        total_train_loss += loss.item()
        tqdm_train_set.set_postfix({'train_loss': total_train_loss / iter_idx})
        iter_idx += 1

    return total_train_loss / (iter_idx-1)

# do one epoch for eval
def eval_epoch():
    tqdm_eval_set = tqdm(eval_set)
    total_eval_loss = 0
    iter_idx = 1
    model.eval()

    # Evaluate data for one epoch
    for batch in tqdm_eval_set:
        with torch.no_grad():
            loss = process_one_batch(batch)
            if loss==None or torch.isnan(loss).item():
                continue
            total_eval_loss += loss.item()
        tqdm_eval_set.set_postfix({'eval_loss': total_eval_loss / iter_idx})
        iter_idx += 1

    return total_eval_loss / (iter_idx-1)

NUM_EPOCHS = 20  # Aumentar a 100 épocas

# train and eval
if __name__ == "__main__":

    # load data
    with open('data_curation/data_training_ready.json') as f:
        print("Loading Data...")
        data = json.load(f)
        train_set, eval_set = split_data(data)
        data = []

    train_set = DataLoader(MyDataset(train_set), batch_size=batch_size, collate_fn=collate_batch, shuffle=True)
    eval_set = DataLoader(MyDataset(eval_set), batch_size=batch_size, collate_fn=collate_batch, shuffle=True)

    lr_scheduler = get_scheduler(
        name="cosine",
        optimizer=optimizer,
        num_warmup_steps=NUM_EPOCHS * len(train_set) / 10,
        num_training_steps=NUM_EPOCHS * len(train_set),
    )
    if LOAD_FROM_CHECKPOINT and os.path.exists('weights3.pth'):
        checkpoint = torch.load('weights3.pth')
        if torch.cuda.device_count() > 1:
            model.module.load_state_dict(checkpoint['model'])
        else:
            model.load_state_dict(checkpoint['model'])
        optimizer.load_state_dict(checkpoint['optimizer'])
        lr_scheduler.load_state_dict(checkpoint['lr_sched'])
        pre_epoch = checkpoint['epoch']
        best_epoch = checkpoint['best_epoch']
        min_eval_loss = checkpoint['min_eval_loss']
        print("Successfully Loaded Checkpoint from Epoch %d" % pre_epoch)

    else:
        pre_epoch = 0
        best_epoch = 0
        min_eval_loss = 100

    model = model.to(device)
    optimizer = torch.optim.AdamW(model.parameters(), lr=LEARNING_RATE)

    for epoch in range(1, NUM_EPOCHS+1-pre_epoch):
        epoch += pre_epoch
        print('-' * 21 + "Epoch " + str(epoch) + '-' * 21)
        train_loss = train_epoch()
        eval_loss = eval_epoch()
        with open('logs.txt','a') as f:
            f.write("Epoch " + str(epoch) + "\ntrain_loss: " + str(train_loss) + "\neval_loss: " +str(eval_loss) + "\ntime: " + time.asctime(time.localtime(time.time())) + "\n\n")
        if eval_loss < min_eval_loss:
            best_epoch = epoch
            min_eval_loss = eval_loss
            if torch.cuda.device_count() > 1:
                checkpoint = {
                'model': model.module.state_dict(),
                'optimizer': optimizer.state_dict(),
                'lr_sched': lr_scheduler.state_dict(),
                'epoch': epoch,
                'best_epoch': best_epoch,
                'min_eval_loss': min_eval_loss}
            else:
                checkpoint = {
                'model': model.state_dict(),
                'optimizer': optimizer.state_dict(),
                'lr_sched': lr_scheduler.state_dict(),
                'epoch': epoch,
                'best_epoch': best_epoch,
                'min_eval_loss': min_eval_loss}
            torch.save(checkpoint, 'weights3.pth')

    print("Best Eval Epoch : "+str(best_epoch))
    print("Min Eval Loss : "+str(min_eval_loss))

Parameter Number: 88425984
Loading Data...


<ipython-input-5-95c0fed4c85f>:39: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = GradScaler()
100%|██████████| 302/302 [00:00<00:00, 20070.98it/s]


---------------------Epoch 1---------------------


  0%|          | 0/2721 [00:00<?, ?it/s]<ipython-input-5-95c0fed4c85f>:95: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():
`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.
100%|██████████| 302/302 [00:04<00:00, 60.61it/s, eval_loss=0.0816]


---------------------Epoch 2---------------------


...

---------------------Epoch 5---------------------


100%|██████████| 302/302 [00:04<00:00, 74.25it/s, eval_loss=0.0732]


---------------------Epoch 6---------------------


  6%|▋         | 174/2721 [00:14<03:39, 11.61it/s, train_loss=0.0758]


KeyboardInterrupt: 

In [ ]:
import time
import torch
import re
import os
from tqdm import tqdm
import requests
import sys
import gc
import numpy as np

# Definir rutas específicas para tu entorno
project_root = '/content/drive/MyDrive/tunesformer-main'
weights_path = f'{project_root}/weights3.pth'
output_dir = f'{project_root}/output_tunes'
prompt_path = f'{project_root}/prompt.txt'

# Asegurarse de que project_root está en sys.path
if project_root not in sys.path:
    sys.path.append(project_root)

# Importar módulos después de configurar sys.path
try:
    from utils import Patchilizer, TunesFormer
    from config import *
    from transformers import GPT2Config
    print("Módulos importados correctamente")
except ImportError as e:
    print(f"Error al importar módulos: {e}")
    sys.exit(1)

# Definir las funciones de muestreo directamente para evitar problemas
def custom_top_p_sampling(probs, p=0.9):
    """
    Implementación de top-p sampling (nucleus sampling)

    Args:
        probs: Probabilidades de cada token
        p: Valor de probabilidad acumulativa para filtrar tokens (0.0-1.0)

    Returns:
        Distribución de probabilidad filtrada
    """
    # Ordena las probabilidades de mayor a menor
    sorted_indices = np.argsort(-probs)
    sorted_probs = probs[sorted_indices]
    cumulative_probs = np.cumsum(sorted_probs)

    # Eliminar tokens con probabilidad acumulada > p
    sorted_indices_to_remove = cumulative_probs > p

    # Desplazar la máscara para mantener al menos un token
    if len(sorted_indices_to_remove) > 1:
        sorted_indices_to_remove[1:] = sorted_indices_to_remove[:-1]
    sorted_indices_to_remove[0] = False

    # Aplicar la máscara
    indices_to_remove = np.zeros_like(probs, dtype=bool)
    for i, should_remove in enumerate(sorted_indices_to_remove):
        if should_remove:
            indices_to_remove[sorted_indices[i]] = True

    # Filtrar y renormalizar
    filtered_probs = np.copy(probs)
    filtered_probs[indices_to_remove] = 0
    sum_p = np.sum(filtered_probs)
    if sum_p > 0:
        filtered_probs = filtered_probs / sum_p
    else:
        # Si todas las probabilidades son cero, usar distribución uniforme
        filtered_probs = np.ones_like(probs) / len(probs)

    return filtered_probs

def custom_top_k_sampling(probs, k=50):
    """
    Implementación de top-k sampling

    Args:
        probs: Probabilidades de cada token
        k: Número de tokens con mayor probabilidad a mantener

    Returns:
        Distribución de probabilidad filtrada
    """
    # Ajusta k si es mayor que el número de tokens
    k = min(k, len(probs))

    # Obtiene los k tokens con mayor probabilidad
    top_k_indices = np.argsort(-probs)[:k]
    top_k_probs = probs[top_k_indices]

    # Crea una nueva distribución
    filtered_probs = np.zeros_like(probs)
    filtered_probs[top_k_indices] = top_k_probs

    # Renormaliza
    filtered_probs = filtered_probs / np.sum(filtered_probs)

    return filtered_probs

def custom_temperature_sampling(probs, temperature=1.0, seed=None):
    """
    Implementación de temperature sampling

    Args:
        probs: Probabilidades de cada token
        temperature: Factor de temperatura (>1 más aleatorio, <1 más determinista)
        seed: Semilla para reproducibilidad

    Returns:
        Índice del token seleccionado
    """
    if seed is not None:
        np.random.seed(seed)

    if temperature == 0:
        # Si temperatura es cero, retornar el token con mayor probabilidad
        return np.argmax(probs)

    # Aplicar temperatura
    if temperature != 1.0:
        probs = np.power(probs, 1.0 / max(1e-8, temperature))
        # Renormalizar
        probs = probs / np.sum(probs)

    # Seleccionar un token basado en la distribución
    return np.random.choice(len(probs), p=probs)

def free_memory():
    """Liberar memoria GPU y RAM"""
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
    gc.collect()

def ensure_dirs_exist(paths):
    """Asegura que los directorios existan"""
    for path in paths:
        # Si es un archivo, obtener el directorio
        if '.' in os.path.basename(path):
            directory = os.path.dirname(path)
        else:
            directory = path

        if directory and not os.path.exists(directory):
            print(f"Creando directorio: {directory}")
            os.makedirs(directory, exist_ok=True)

# Modificación de la clase TunesFormer para usar nuestras funciones personalizadas
class CustomTunesFormer(TunesFormer):
    def generate(self, patches, tokens=None, top_p=1, top_k=0, temperature=1, seed=None):
        """
        Versión personalizada del método generate que usa nuestras funciones de muestreo
        """
        patches = patches.reshape(len(patches), -1, PATCH_SIZE)
        encoded_patches = self.patch_level_decoder(patches)["last_hidden_state"]
        if tokens is None:
            tokens = torch.tensor([self.bos_token_id], device=self.device)
        generated_patch = []

        if seed is not None:
            import random
            random.seed(seed)

        while True:
            if seed is not None:
                n_seed = random.randint(0, 1000000)
                random.seed(n_seed)
            else:
                n_seed = None

            # Obtener probabilidades
            prob = self.char_level_decoder.generate(encoded_patches[0][-1], tokens).cpu().detach().numpy()

            # Aplicar muestreos personalizados
            prob = custom_top_p_sampling(prob, p=top_p)
            prob = custom_top_k_sampling(prob, k=top_k if top_k > 0 else len(prob))
            token = custom_temperature_sampling(prob, temperature=temperature, seed=n_seed)

            generated_patch.append(token)
            if token == self.eos_token_id or len(tokens) >= PATCH_SIZE - 1:
                break
            else:
                tokens = torch.cat((tokens, torch.tensor([token], device=self.device)), dim=0)

        return generated_patch, n_seed

def generate_abc(num_tunes=3, max_patch=128, top_p=0.8, top_k=8,
                 temperature=1.2, seed=None, show_control_code=True,
                 weights=weights_path, prompt=prompt_path, output=output_dir):
    """Genera melodías ABC utilizando el modelo TunesFormer"""

    # Asegurar que los directorios existan
    ensure_dirs_exist([weights, output, prompt])

    # Verificar disponibilidad de GPU
    if torch.cuda.is_available():
        device = torch.device("cuda")
        print(f"Usando GPU: {torch.cuda.get_device_name(0)}")
    else:
        device = torch.device("cpu")
        print("Usando CPU (el proceso será más lento)")

    # Inicializar modelo y componentes
    try:
        patchilizer = Patchilizer()

        # Configuración del modelo
        patch_config = GPT2Config(num_hidden_layers=PATCH_NUM_LAYERS,
                            max_length=PATCH_LENGTH,
                            max_position_embeddings=PATCH_LENGTH,
                            vocab_size=1)
        char_config = GPT2Config(num_hidden_layers=CHAR_NUM_LAYERS,
                            max_length=PATCH_SIZE,
                            max_position_embeddings=PATCH_SIZE,
                            vocab_size=128)

        # Usar nuestra clase personalizada en lugar de TunesFormer
        model = CustomTunesFormer(patch_config, char_config, share_weights=SHARE_WEIGHTS)

        print(f"Modelo inicializado con {sum(p.numel() for p in model.parameters() if p.requires_grad)} parámetros")
    except Exception as e:
        print(f"Error al inicializar el modelo: {e}")
        return

    # Cargar pesos del modelo
    if os.path.exists(weights):
        print(f"Cargando pesos desde '{weights}'...")
        try:
            # Algunas veces hay problemas de memoria, intentar liberar primero
            free_memory()

            # Cargar checkpoint con map_location para flexibilidad GPU/CPU
            checkpoint = torch.load(weights, map_location=device)

            # Diferentes formas de cargar según la estructura del checkpoint
            if 'model' in checkpoint:
                model.load_state_dict(checkpoint['model'])
            else:
                model.load_state_dict(checkpoint)

            print("Pesos cargados exitosamente")
        except Exception as e:
            print(f"Error al cargar pesos: {e}")
            print("Intentando descargar pesos desde Hugging Face...")
            try:
                url = 'https://huggingface.co/sander-wood/tunesformer/resolve/main/weights.pth'
                response = requests.get(url, stream=True)

                total_size = int(response.headers.get('content-length', 0))
                chunk_size = 1024

                with open(weights, 'wb') as file, tqdm(
                    desc=os.path.basename(weights),
                    total=total_size,
                    unit='B',
                    unit_scale=True,
                    unit_divisor=1024,
                ) as bar:
                    for data in response.iter_content(chunk_size=chunk_size):
                        size = file.write(data)
                        bar.update(size)

                # Intentar cargar de nuevo
                checkpoint = torch.load(weights, map_location=device)
                if 'model' in checkpoint:
                    model.load_state_dict(checkpoint['model'])
                else:
                    model.load_state_dict(checkpoint)
                print("Pesos descargados y cargados exitosamente")
            except Exception as e:
                print(f"Error al descargar/cargar pesos: {e}")
                return
    else:
        print(f"No se encontraron pesos en '{weights}'. Descargando...")
        try:
            url = 'https://huggingface.co/sander-wood/tunesformer/resolve/main/weights.pth'
            response = requests.get(url, stream=True)

            total_size = int(response.headers.get('content-length', 0))
            chunk_size = 1024

            with open(weights, 'wb') as file, tqdm(
                desc=os.path.basename(weights),
                total=total_size,
                unit='B',
                unit_scale=True,
                unit_divisor=1024,
            ) as bar:
                for data in response.iter_content(chunk_size=chunk_size):
                    size = file.write(data)
                    bar.update(size)

            # Cargar los pesos descargados
            checkpoint = torch.load(weights, map_location=device)
            if 'model' in checkpoint:
                model.load_state_dict(checkpoint['model'])
            else:
                model.load_state_dict(checkpoint)
            print("Pesos descargados y cargados exitosamente")
        except Exception as e:
            print(f"Error: {e}")
            return

    # Mover modelo al dispositivo apropiado y configurar para evaluación
    model = model.to(device)
    model.eval()

    # Cargar prompt
    try:
        if os.path.exists(prompt):
            with open(prompt, 'r', encoding='utf-8') as f:
                prompt_text = f.read()
                print(f"Prompt cargado desde '{prompt}'")
        else:
            # Si no existe el archivo, usar un prompt por defecto
            prompt_text = """M:3/4
L:1/8
K:C
"""
            # Guardar el prompt por defecto para futuros usos
            with open(prompt, 'w', encoding='utf-8') as f:
                f.write(prompt_text)
            print(f"Archivo de prompt no encontrado. Se creó un prompt por defecto en '{prompt}'")
    except Exception as e:
        print(f"Error al cargar el prompt: {e}")
        # Usar prompt mínimo
        prompt_text = """M:3/4
L:1/8
K:C
"""
        print("Se usará un prompt mínimo por defecto")

    # Inicializar variable para almacenar todas las melodías
    tunes = ""

    # Mostrar parámetros de generación
    print("\n" + " PARÁMETROS DE GENERACIÓN ".center(60, "#") + '\n')
    params = {
        'num_tunes': num_tunes,
        'max_patch': max_patch,
        'top_p': top_p,
        'top_k': top_k,
        'temperature': temperature,
        'seed': seed,
        'show_control_code': show_control_code
    }
    for key, value in params.items():
        print(f"{key}: {value}")
    print("\n" + " MELODÍAS GENERADAS ".center(60, "#"))

    # Tiempo de inicio para medir la generación
    start_time = time.time()

    try:
        # Generar cada melodía
        for i in range(num_tunes):
            print(f"\n--- Melodía {i+1}/{num_tunes} ---\n")

            # Preparar la melodía con el número y el prompt
            tune = f"X:{i+1}\n{prompt_text}"
            lines = re.split(r'(\n)', tune)
            tune_output = ""
            skip = False

            # Procesar y mostrar líneas iniciales
            for line in lines:
                if show_control_code or line[:2] not in ["S:", "B:", "E:"]:
                    if not skip:
                        print(line, end="")
                        tune_output += line
                    skip = False
                else:
                    skip = True

            # Preparar entrada para el modelo
            try:
                input_patches = torch.tensor([patchilizer.encode(prompt_text, add_special_patches=True)[:-1]], device=device)

                if tune_output == "":
                    tokens = None
                else:
                    prefix = patchilizer.decode(input_patches[0])
                    remaining_tokens = prompt_text[len(prefix):]
                    tokens = torch.tensor([patchilizer.bos_token_id] + [ord(c) for c in remaining_tokens], device=device)

                # Generar la melodía secuencialmente hasta alcanzar max_patch o EOS
                while input_patches.shape[1] < max_patch:
                    # Liberar memoria periódicamente
                    if input_patches.shape[1] % 20 == 0:
                        free_memory()

                    # Generar siguiente patch
                    predicted_patch, seed = model.generate(
                        input_patches,
                        tokens,
                        top_p=top_p,
                        top_k=top_k,
                        temperature=temperature,
                        seed=seed
                    )
                    tokens = None

                    # Si no es fin de secuencia, procesar el nuevo patch
                    if predicted_patch[0] != patchilizer.eos_token_id:
                        next_bar = patchilizer.decode([predicted_patch])

                        # Mostrar y agregar a la salida si no es código de control o si se solicita mostrarlos
                        if show_control_code or next_bar[:2] not in ["S:", "B:", "E:"]:
                            print(next_bar, end="")
                            tune_output += next_bar

                        # Si el patch generado está vacío, terminar
                        if next_bar == "":
                            break

                        # Preparar para la siguiente iteración
                        next_bar = remaining_tokens + next_bar
                        remaining_tokens = ""
                        predicted_patch = torch.tensor(patchilizer.bar2patch(next_bar), device=device).unsqueeze(0)

                        # Concatenar a las entradas
                        input_patches = torch.cat([input_patches, predicted_patch.unsqueeze(0)], dim=1)
                    else:
                        # Fin de secuencia encontrado
                        break

                # Añadir la melodía completa a la colección
                tunes += tune_output + "\n\n"
                print("\n")

            except RuntimeError as e:
                # Manejo específico para errores de memoria
                if "memory" in str(e).lower():
                    print(f"Error de memoria al generar melodía {i+1}: {e}")
                    print("Intentando liberar memoria y continuar con la siguiente melodía...")
                    free_memory()
                else:
                    print(f"Error al generar melodía {i+1}: {e}")

                # Añadir lo que se haya generado hasta ahora
                tunes += tune_output + "\n\n"
                print("\n")
    except Exception as e:
        print(f"Error durante la generación: {e}")
        import traceback
        traceback.print_exc()

    # Mostrar tiempo total de generación
    generation_time = time.time() - start_time
    print(f"Tiempo de generación: {generation_time:.2f} segundos")

    # Crear timestamp para el nombre de archivo
    timestamp = time.strftime("%a_%d_%b_%Y_%H_%M_%S", time.localtime())

    # Asegurar que el directorio de salida existe
    os.makedirs(output, exist_ok=True)

    # Guardar las melodías generadas
    output_file = os.path.join(output, f'{timestamp}.abc')
    try:
        with open(output_file, 'w', encoding='utf-8') as f:
            f.write(tunes)
        print(f"Melodías guardadas en: {output_file}")
    except Exception as e:
        print(f"Error al guardar el archivo: {e}")
        # Intentar guardar en ubicación alternativa
        alt_file = f'./output_{timestamp}.abc'
        try:
            with open(alt_file, 'w', encoding='utf-8') as f:
                f.write(tunes)
            print(f"Melodías guardadas en ubicación alternativa: {alt_file}")
        except:
            print("No se pudieron guardar las melodías generadas")

    return tunes

# Ejemplo de uso:
# Puedes ejecutar la función directamente sin parámetros para usar valores predeterminados
# generate_abc()

# O con parámetros personalizados
# generate_abc(num_tunes=5, temperature=1.3, top_p=0.95, top_k=10)

Módulos importados correctamente


In [ ]:
generate_abc(num_tunes=1, temperature=0.1, top_p=0.95, top_k=10)

Usando GPU: Tesla T4
Modelo inicializado con 88425984 parámetros
Cargando pesos desde '/content/drive/MyDrive/tunesformer-main/weights3.pth'...
Error al cargar pesos: Error(s) in loading state_dict for CustomTunesFormer:
	Unexpected key(s) in state_dict: "patch_level_decoder.base.h.0.attn.bias", "patch_level_decoder.base.h.0.attn.masked_bias", "patch_level_decoder.base.h.1.attn.bias", "patch_level_decoder.base.h.1.attn.masked_bias", "patch_level_decoder.base.h.2.attn.bias", "patch_level_decoder.base.h.2.attn.masked_bias", "patch_level_decoder.base.h.3.attn.bias", "patch_level_decoder.base.h.3.attn.masked_bias", "patch_level_decoder.base.h.4.attn.bias", "patch_level_decoder.base.h.4.attn.masked_bias", "patch_level_decoder.base.h.5.attn.bias", "patch_level_decoder.base.h.5.attn.masked_bias", "patch_level_decoder.base.h.6.attn.bias", "patch_level_decoder.base.h.6.attn.masked_bias", "patch_level_decoder.base.h.7.attn.bias", "patch_level_decoder.base.h.7.attn.masked_bias", "patch_level_deco

weights3.pth: 100%|██████████| 0.99G/0.99G [00:14<00:00, 75.1MB/s]


Error al descargar/cargar pesos: Error(s) in loading state_dict for CustomTunesFormer:
	Unexpected key(s) in state_dict: "patch_level_decoder.base.h.0.attn.bias", "patch_level_decoder.base.h.0.attn.masked_bias", "patch_level_decoder.base.h.1.attn.bias", "patch_level_decoder.base.h.1.attn.masked_bias", "patch_level_decoder.base.h.2.attn.bias", "patch_level_decoder.base.h.2.attn.masked_bias", "patch_level_decoder.base.h.3.attn.bias", "patch_level_decoder.base.h.3.attn.masked_bias", "patch_level_decoder.base.h.4.attn.bias", "patch_level_decoder.base.h.4.attn.masked_bias", "patch_level_decoder.base.h.5.attn.bias", "patch_level_decoder.base.h.5.attn.masked_bias", "patch_level_decoder.base.h.6.attn.bias", "patch_level_decoder.base.h.6.attn.masked_bias", "patch_level_decoder.base.h.7.attn.bias", "patch_level_decoder.base.h.7.attn.masked_bias", "patch_level_decoder.base.h.8.attn.bias", "patch_level_decoder.base.h.8.attn.masked_bias", "char_level_decoder.base.transformer.h.0.attn.bias", "char_l

In [ ]:
import time
import torch
import os
import sys
import numpy as np
import argparse
import re
from tqdm import tqdm

# Configurar CUDA para mejor debugging
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

# Importar después de configurar CUDA
from utils import *
from config import *
from transformers import GPT2Config

def get_args(parser):
    parser.add_argument('-num_tunes', type=int, default=1, help='the number of independently computed returned tunes')
    parser.add_argument('-max_patch', type=int, default=16, help='integer to define the maximum length in tokens of each tune')
    parser.add_argument('-top_p', type=float, default=0.9, help='float to define the tokens that are within the sample operation of text generation')
    parser.add_argument('-top_k', type=int, default=10, help='integer to define the tokens that are within the sample operation of text generation')
    parser.add_argument('-temperature', type=float, default=0.8, help='the temperature of the sampling operation')
    parser.add_argument('-seed', type=int, default=42, help='seed for randomstate')
    parser.add_argument('-show_control_code', type=bool, default=False, help='whether to show control code')
    parser.add_argument('-voice', type=str, default='V1', choices=['V1', 'V2', 'V3', 'V4'], help='voice to generate')
    parser.add_argument('-key', type=str, default='C', help='key signature')
    parser.add_argument('-meter', type=str, default='4/4', help='time signature')
    parser.add_argument('-weights', type=str, default='weights3.pth', help='path to model weights')

    # Fix para Colab
    args, unknown = parser.parse_known_args()
    return args

def create_simple_prompt(voice='V1', key='C', meter='4/4'):
    """Crea un prompt muy simple para evitar problemas"""
    # Prompt mínimo basado en tus datos de entrenamiento
    control_code = f"PIECE:gen123 VOICE:{voice} MEASURES:8 KEY:{key} METER:{meter} COMPLEXITY:SIMPLE"
    abc_header = f"X:1\nM:{meter}\nK:{key}\nV:{voice[-1]}"

    return control_code + "\n" + abc_header

def safe_model_load(model, checkpoint_path, device):
    """Carga el modelo de manera segura"""
    try:
        print(f"Loading checkpoint from: {checkpoint_path}")

        # Cargar en CPU primero
        checkpoint = torch.load(checkpoint_path, map_location='cpu')

        # Verificar estructura del checkpoint
        if 'model' in checkpoint:
            state_dict = checkpoint['model']
            print(f"Checkpoint info - Epoch: {checkpoint.get('epoch', 'unknown')}")
            if 'config' in checkpoint:
                print(f"Saved config: {checkpoint['config']}")
        else:
            state_dict = checkpoint

        # Verificar compatibilidad básica
        model_keys = set(model.state_dict().keys())
        checkpoint_keys = set(state_dict.keys())

        if model_keys != checkpoint_keys:
            print("Warning: Model structure mismatch")
            missing_keys = model_keys - checkpoint_keys
            unexpected_keys = checkpoint_keys - model_keys
            if missing_keys:
                print(f"Missing keys: {list(missing_keys)[:5]}...")
            if unexpected_keys:
                print(f"Unexpected keys: {list(unexpected_keys)[:5]}...")

        # Cargar state dict
        model.load_state_dict(state_dict, strict=False)

        # Mover a device después de cargar
        model = model.to(device)
        model.eval()

        print("✅ Model loaded successfully!")
        return model

    except Exception as e:
        print(f"❌ Error loading model: {e}")
        return None

def generate_abc(args):
    # Configurar device
    if torch.cuda.is_available():
        device = torch.device("cuda")
        print(f"Using GPU: {torch.cuda.get_device_name()}")
        # Limpiar cache de CUDA
        torch.cuda.empty_cache()
    else:
        device = torch.device("cpu")
        print("Using CPU")

    try:
        # Crear patchilizer
        patchilizer = Patchilizer()
        print("✅ Patchilizer created")

        # Configuración del modelo - usar valores más seguros
        print("Creating model configuration...")

        # Valores por defecto más conservadores
        patch_layers = getattr(config, 'PATCH_NUM_LAYERS', 6)
        char_layers = getattr(config, 'CHAR_NUM_LAYERS', 6)
        patch_length = getattr(config, 'PATCH_LENGTH', 128)
        patch_size = getattr(config, 'PATCH_SIZE', 32)
        share_weights = getattr(config, 'SHARE_WEIGHTS', True)

        print(f"Model config: patch_layers={patch_layers}, char_layers={char_layers}")
        print(f"Patch config: length={patch_length}, size={patch_size}")

        patch_config = GPT2Config(
            num_hidden_layers=patch_layers,
            max_length=patch_length,
            max_position_embeddings=patch_length,
            vocab_size=1
        )

        char_config = GPT2Config(
            num_hidden_layers=char_layers,
            max_length=patch_size,
            max_position_embeddings=patch_size,
            vocab_size=128
        )

        # Crear modelo
        print("Creating TunesFormer model...")
        model = TunesFormer(patch_config, char_config, share_weights=share_weights)
        print("✅ Model created")

        # Cargar pesos
        if not os.path.exists(args.weights):
            print(f"❌ Weights file '{args.weights}' not found!")
            return

        model = safe_model_load(model, args.weights, device)
        if model is None:
            return

        # Crear prompt simple
        prompt = create_simple_prompt(args.voice, args.key, args.meter)
        print(f"Using prompt:\n{prompt}\n")

        # Parámetros de generación
        max_patch = min(args.max_patch, 20)  # Limitar para evitar problemas

        print(" GENERATION PARAMETERS ".center(60, "#"))
        print(f"max_patch: {max_patch}")
        print(f"temperature: {args.temperature}")
        print(f"top_p: {args.top_p}")
        print(f"top_k: {args.top_k}")
        print("\n" + " OUTPUT ".center(60, "#"))

        all_tunes = []
        start_time = time.time()

        for i in range(args.num_tunes):
            print(f"\n--- Generating Tune {i+1}/{args.num_tunes} ---")

            try:
                # Mostrar prompt inicial
                if args.show_control_code:
                    display_text = prompt
                else:
                    # Solo mostrar la parte ABC
                    lines = prompt.split('\n')
                    abc_start = next((j for j, line in enumerate(lines) if line.startswith('X:')), 0)
                    display_text = '\n'.join(lines[abc_start:])

                print("Starting with:")
                print(display_text)
                print("\nGenerating...")

                # Codificar prompt de manera segura
                try:
                    with torch.no_grad():
                        encoded = patchilizer.encode(prompt, add_special_patches=True)

                        if len(encoded) == 0:
                            print("❌ Empty encoding")
                            continue

                        # Preparar input patches
                        if len(encoded) > 1:
                            encoded = encoded[:-1]  # Quitar último token

                        input_patches = torch.tensor([encoded], device=device, dtype=torch.long)
                        print(f"Input shape: {input_patches.shape}")

                except Exception as e:
                    print(f"❌ Encoding error: {e}")
                    continue

                # Generación paso a paso
                generated_text = display_text

                for step in range(max_patch):
                    try:
                        with torch.no_grad():
                            # Generación simple
                            outputs = model(input_patches)

                            if hasattr(outputs, 'logits'):
                                logits = outputs.logits[:, -1, :]  # Último token

                                # Sampling manual simple
                                if args.temperature > 0:
                                    logits = logits / args.temperature

                                probs = torch.softmax(logits, dim=-1)
                                next_token = torch.multinomial(probs, num_samples=1)

                                # Verificar si es EOS
                                if hasattr(patchilizer, 'eos_token_id') and next_token.item() == patchilizer.eos_token_id:
                                    print("\n[EOS reached]")
                                    break

                                # Decodificar
                                try:
                                    decoded = patchilizer.decode([next_token.item()])
                                    if decoded:
                                        print(decoded, end="", flush=True)
                                        generated_text += decoded

                                        # Actualizar input para siguiente iteración
                                        input_patches = torch.cat([input_patches, next_token.unsqueeze(0)], dim=1)
                                    else:
                                        break

                                except Exception as e:
                                    print(f"\n[Decode error: {e}]")
                                    break
                            else:
                                print("\n[No logits in output]")
                                break

                    except Exception as e:
                        print(f"\n[Generation error at step {step}: {e}]")
                        break

                all_tunes.append(generated_text)
                print(f"\n\n✅ Tune {i+1} completed!")

            except Exception as e:
                print(f"❌ Error generating tune {i+1}: {e}")
                continue

        # Guardar resultados
        print(f"\nGeneration time: {time.time()-start_time:.2f} seconds")

        if all_tunes:
            timestamp = time.strftime("%Y%m%d_%H%M%S", time.localtime())
            os.makedirs('output_tunes', exist_ok=True)

            output_file = f'output_tunes/tune_{timestamp}_{args.voice}_{args.key}.abc'
            with open(output_file, 'w', encoding='utf-8') as f:
                f.write('\n\n'.join(all_tunes))

            print(f"✅ Saved {len(all_tunes)} tune(s) to: {output_file}")

            # Mostrar resultado
            print("\n" + "="*50)
            print("GENERATED MUSIC:")
            print("="*50)
            result = all_tunes[0]
            print(result[:300] + ("..." if len(result) > 300 else ""))
        else:
            print("❌ No tunes generated successfully")

    except Exception as e:
        print(f"❌ Fatal error: {e}")
        import traceback
        traceback.print_exc()

if __name__ == "__main__":
    parser = argparse.ArgumentParser(description='Generate music with TunesFormer')
    args = get_args(parser)

    print("🎵 TunesFormer Music Generator (Safe Mode)")
    print("="*60)

    try:
        generate_abc(args)
    except KeyboardInterrupt:
        print("\n⏹️ Generation interrupted by user")
    except Exception as e:
        print(f"💥 Unexpected error: {e}")
        import traceback
        traceback.print_exc()

🎵 TunesFormer Music Generator (Safe Mode)
Using GPU: Tesla T4
✅ Patchilizer created
Creating model configuration...
❌ Fatal error: name 'config' is not defined


Traceback (most recent call last):
  File "<ipython-input-7-fc16222eb7a8>", line 107, in generate_abc
    patch_layers = getattr(config, 'PATCH_NUM_LAYERS', 6)
                           ^^^^^^
NameError: name 'config' is not defined


Nueva forma para entrenar modelo, aún no provada

In [ ]:
import os
import time
import json
import torch
import random
import numpy as np
from utils import *
from config import *
from tqdm import tqdm
from torch.cuda.amp import autocast, GradScaler
from torch.utils.data import Dataset, DataLoader
from transformers import GPT2Config, get_scheduler

# Fix para determinismo
def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

set_seed(42)

if torch.cuda.is_available():
    device = torch.device("cuda")
    print(f"Using GPU: {torch.cuda.get_device_name()}")
else:
    device = torch.device("cpu")

# CONFIGURACIÓN OPTIMIZADA
BATCH_SIZE = 4  # Batch size fijo y mayor
LEARNING_RATE = 1e-5  # Learning rate más conservador
GRADIENT_CLIP = 1.0  # Gradient clipping
WEIGHT_DECAY = 0.01  # Weight decay para regularización
WARMUP_RATIO = 0.1  # 10% de warmup
EVAL_STEPS = 100  # Evaluar cada 100 steps
SAVE_STEPS = 500  # Guardar cada 500 steps
PATIENCE = 5  # Early stopping patience

patchilizer = Patchilizer()

patch_config = GPT2Config(
    num_hidden_layers=PATCH_NUM_LAYERS,
    max_length=PATCH_LENGTH,
    max_position_embeddings=PATCH_LENGTH,
    vocab_size=1,
    hidden_dropout_prob=0.1,  # Añadir dropout
    attention_probs_dropout_prob=0.1
)

char_config = GPT2Config(
    num_hidden_layers=CHAR_NUM_LAYERS,
    max_length=PATCH_SIZE,
    max_position_embeddings=PATCH_SIZE,
    vocab_size=128,
    hidden_dropout_prob=0.1,  # Añadir dropout
    attention_probs_dropout_prob=0.1
)

model = TunesFormer(patch_config, char_config, share_weights=SHARE_WEIGHTS)

# Print parameter number
total_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"Parameter Number: {total_params:,}")

if torch.cuda.device_count() > 1:
    print(f"Using {torch.cuda.device_count()} GPUs")
    model = torch.nn.DataParallel(model)

scaler = GradScaler()
optimizer = torch.optim.AdamW(
    model.parameters(),
    lr=LEARNING_RATE,
    weight_decay=WEIGHT_DECAY,
    betas=(0.9, 0.999),
    eps=1e-8
)

def collate_batch(batch):
    input_patches = []

    for input_patch in batch:
        input_patches.append(input_patch.reshape(-1))

    input_patches = torch.nn.utils.rnn.pad_sequence(
        input_patches, batch_first=True, padding_value=0
    )

    return input_patches.to(device)

def split_data(data, eval_ratio=0.15):  # Más datos para evaluación
    random.shuffle(data)
    split_idx = int(len(data) * eval_ratio)
    eval_set = data[:split_idx]
    train_set = data[split_idx:]
    return train_set, eval_set

class MyDataset(Dataset):
    def __init__(self, items, max_length=None):
        self.texts = []
        failed_items = 0

        print(f"Processing {len(items)} items...")

        for item in tqdm(items, desc="Encoding data"):
            # FIX: No remover la primera línea de ABC notation
            control_code = item['control code'].strip()
            abc_notation = item['abc notation'].strip()

            # Combinar correctamente
            text = control_code + "\n" + abc_notation

            try:
                input_patch = patchilizer.encode(text, add_special_patches=True)
                input_patch = torch.tensor(input_patch, dtype=torch.long)

                # Filtrar secuencias muy cortas o muy largas
                if len(input_patch) < 10:
                    failed_items += 1
                    continue

                if max_length and len(input_patch) > max_length:
                    input_patch = input_patch[:max_length]

                if torch.sum(input_patch) != 0:
                    self.texts.append(input_patch)
                else:
                    failed_items += 1

            except Exception as e:
                failed_items += 1
                continue

        print(f"Successfully processed: {len(self.texts)} items")
        print(f"Failed items: {failed_items}")

        if len(self.texts) == 0:
            raise ValueError("No valid training data found!")

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        return self.texts[idx]

def process_one_batch(batch):
    input_patches = batch
    outputs = model(input_patches)
    loss = outputs.loss
    return loss.mean() if loss is not None else None

def train_epoch(train_loader, eval_loader, epoch):
    model.train()
    total_train_loss = 0
    num_batches = 0
    step = 0

    progress_bar = tqdm(train_loader, desc=f"Epoch {epoch}")

    for batch in progress_bar:
        try:
            # Forward pass con mixed precision
            with autocast():
                loss = process_one_batch(batch)

            if loss is None or torch.isnan(loss):
                continue

            # Backward pass
            scaler.scale(loss).backward()

            # Gradient clipping
            scaler.unscale_(optimizer)
            torch.nn.utils.clip_grad_norm_(model.parameters(), GRADIENT_CLIP)

            # Optimizer step
            scaler.step(optimizer)
            scaler.update()
            lr_scheduler.step()

            model.zero_grad()

            total_train_loss += loss.item()
            num_batches += 1
            step += 1

            # Update progress bar
            avg_loss = total_train_loss / num_batches
            progress_bar.set_postfix({
                'loss': f'{avg_loss:.4f}',
                'lr': f'{lr_scheduler.get_last_lr()[0]:.2e}'
            })

            # Evaluación periódica
            if step % EVAL_STEPS == 0:
                eval_loss = eval_epoch(eval_loader, verbose=False)
                model.train()  # Volver a training mode

                print(f"\nStep {step} - Train Loss: {avg_loss:.4f}, Eval Loss: {eval_loss:.4f}")

        except RuntimeError as e:
            if "memory" in str(e):
                print(f"CUDA OOM: {e}")
                torch.cuda.empty_cache()
                continue
            else:
                raise e

    return total_train_loss / max(num_batches, 1)

def eval_epoch(eval_loader, verbose=True):
    model.eval()
    total_eval_loss = 0
    num_batches = 0

    progress_bar = tqdm(eval_loader, desc="Evaluating") if verbose else eval_loader

    with torch.no_grad():
        for batch in progress_bar:
            try:
                loss = process_one_batch(batch)

                if loss is None or torch.isnan(loss):
                    continue

                total_eval_loss += loss.item()
                num_batches += 1

                if verbose:
                    avg_loss = total_eval_loss / num_batches
                    progress_bar.set_postfix({'eval_loss': f'{avg_loss:.4f}'})

            except RuntimeError as e:
                if "memory" in str(e):
                    torch.cuda.empty_cache()
                    continue
                else:
                    raise e

    return total_eval_loss / max(num_batches, 1)

# CONFIGURACIÓN DE ENTRENAMIENTO
NUM_EPOCHS = 50
LOAD_FROM_CHECKPOINT = True
CHECKPOINT_PATH = 'weights_optimized.pth'

if __name__ == "__main__":
    # Cargar y validar datos
    print("Loading data...")

    # Verificar que el archivo existe
    data_file = 'data_curation/data_training_ready.json'
    if not os.path.exists(data_file):
        print(f"Data file {data_file} not found!")
        exit(1)

    with open(data_file, 'r', encoding='utf-8') as f:
        data = json.load(f)

    print(f"Total data items: {len(data)}")

    # Verificar muestra de datos
    if len(data) > 0:
        sample = data[0]
        print(f"Sample control code: {sample['control code'][:100]}...")
        print(f"Sample ABC notation: {sample['abc notation'][:100]}...")

    # Dividir datos
    train_data, eval_data = split_data(data)
    print(f"Train items: {len(train_data)}, Eval items: {len(eval_data)}")

    # Crear datasets y dataloaders
    train_dataset = MyDataset(train_data, max_length=PATCH_LENGTH)
    eval_dataset = MyDataset(eval_data, max_length=PATCH_LENGTH)

    train_loader = DataLoader(
        train_dataset,
        batch_size=BATCH_SIZE,
        collate_fn=collate_batch,
        shuffle=True,
        num_workers=2,
        pin_memory=True
    )
    eval_loader = DataLoader(
        eval_dataset,
        batch_size=BATCH_SIZE,
        collate_fn=collate_batch,
        shuffle=False,
        num_workers=2,
        pin_memory=True
    )

    print(f"Train batches: {len(train_loader)}, Eval batches: {len(eval_loader)}")

    # Configurar learning rate scheduler
    total_steps = NUM_EPOCHS * len(train_loader)
    warmup_steps = int(total_steps * WARMUP_RATIO)

    lr_scheduler = get_scheduler(
        name="cosine",
        optimizer=optimizer,
        num_warmup_steps=warmup_steps,
        num_training_steps=total_steps,
    )

    # Cargar checkpoint si existe
    start_epoch = 0
    best_eval_loss = float('inf')
    patience_counter = 0

    if LOAD_FROM_CHECKPOINT and os.path.exists(CHECKPOINT_PATH):
        print(f"Loading checkpoint from {CHECKPOINT_PATH}")
        checkpoint = torch.load(CHECKPOINT_PATH, map_location=device)

        if torch.cuda.device_count() > 1:
            model.module.load_state_dict(checkpoint['model'])
        else:
            model.load_state_dict(checkpoint['model'])

        optimizer.load_state_dict(checkpoint['optimizer'])
        lr_scheduler.load_state_dict(checkpoint['lr_scheduler'])
        start_epoch = checkpoint['epoch'] + 1
        best_eval_loss = checkpoint['best_eval_loss']

        print(f"Resumed from epoch {start_epoch}, best eval loss: {best_eval_loss:.4f}")

    # Mover modelo a device
    model = model.to(device)

    # Loop de entrenamiento
    print("Starting training...")

    for epoch in range(start_epoch, NUM_EPOCHS):
        print(f"\n{'='*50}")
        print(f"Epoch {epoch+1}/{NUM_EPOCHS}")
        print(f"{'='*50}")

        # Entrenar
        train_loss = train_epoch(train_loader, eval_loader, epoch+1)

        # Evaluar
        eval_loss = eval_epoch(eval_loader)

        # Log
        print(f"\nEpoch {epoch+1} Results:")
        print(f"Train Loss: {train_loss:.4f}")
        print(f"Eval Loss: {eval_loss:.4f}")
        print(f"Learning Rate: {lr_scheduler.get_last_lr()[0]:.2e}")

        # Guardar logs
        with open('training_logs_optimized.txt', 'a') as f:
            f.write(f"Epoch {epoch+1}\n")
            f.write(f"Train Loss: {train_loss:.4f}\n")
            f.write(f"Eval Loss: {eval_loss:.4f}\n")
            f.write(f"Time: {time.asctime()}\n\n")

        # Guardar mejor modelo
        if eval_loss < best_eval_loss:
            best_eval_loss = eval_loss
            patience_counter = 0

            checkpoint = {
                'epoch': epoch,
                'model': model.module.state_dict() if torch.cuda.device_count() > 1 else model.state_dict(),
                'optimizer': optimizer.state_dict(),
                'lr_scheduler': lr_scheduler.state_dict(),
                'train_loss': train_loss,
                'best_eval_loss': best_eval_loss,
                'config': {
                    'PATCH_NUM_LAYERS': PATCH_NUM_LAYERS,
                    'CHAR_NUM_LAYERS': CHAR_NUM_LAYERS,
                    'PATCH_LENGTH': PATCH_LENGTH,
                    'PATCH_SIZE': PATCH_SIZE
                }
            }

            torch.save(checkpoint, CHECKPOINT_PATH)
            print(f"✅ New best model saved! Eval loss: {eval_loss:.4f}")
        else:
            patience_counter += 1
            print(f"⚠️  No improvement. Patience: {patience_counter}/{PATIENCE}")

        # Early stopping
        if patience_counter >= PATIENCE:
            print(f"🛑 Early stopping triggered after {PATIENCE} epochs without improvement")
            break

    print(f"\n🏆 Training completed!")
    print(f"Best Eval Loss: {best_eval_loss:.4f}")
    print(f"Model saved at: {CHECKPOINT_PATH}")